# 概要
Tensorflow, Kerasによる画像分類の方法について説明します。<br> 
このnotebookでは簡単なEDAとモデルの学習を行います。<br> 

### 1. 準備 
- ファイル構成
- ライブラリのインポート
- 設定

### 2. データ理解
- データの読み込み
- 画像データの可視化
- 各画像に対してのラベル比率の確認

### 3. モデリング
- データ分割方法の定義
- データ前処理の定義
- 分類モデルの定義
- モデルの学習（層化k分割交差検証）
 - 損失関数の定義<br>
 - 最適化手法の定義<br>
 - スコアの算出<br>

### 4. 推論
https://www.kaggle.com/takuyatone/cassava-keras-tf-baseline-inference

# 1. 準備 

## ファイル構成

In [ ]:
!ls /kaggle/input/cassava-leaf-disease-classification

## ライブラリのインポート

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import cv2

print('tensorflow version:', tf.__version__)

In [ ]:
# seed固定
def seed_everything(seed=1234):
    #random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(seed=42)

In [ ]:
# GPUの確認
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## 設定

In [ ]:
class CFG:
    debug=True
    size=64
    epochs=10
    batch_size=64
    val_batch_size=128
    seed=42
    target_size=5
    target_col='label'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]

#  2. データ理解

## データの読み込み

In [ ]:
if os.path.exists('/kaggle/input'):
    # kaggle環境
    DATA_DIR = '/kaggle/input/cassava-leaf-disease-classification/'
else:
    # ローカル環境
    DATA_DIR = '../../data/raw/'
    
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'sample_submission.csv')
label_map = pd.read_json(DATA_DIR + 'label_num_to_disease_map.json', 
                         orient='index')
display(train.head())
display(test.head())
display(label_map)

## 画像データの可視化

In [ ]:
# 画像の読み込み
image = plt.imread(DATA_DIR + 'train_images/1000015157.jpg')

# 画像の表示
plt.imshow(image);

In [ ]:
label0_images = train[train['label'] == 0]['image_id'].to_list()
label1_images = train[train['label'] == 1]['image_id'].to_list()
label2_images = train[train['label'] == 2]['image_id'].to_list()
label3_images = train[train['label'] == 3]['image_id'].to_list()
label4_images = train[train['label'] == 4]['image_id'].to_list()

In [ ]:
def showImages(images):

    random_images = [np.random.choice(images) for i in range(8)]

    plt.figure(figsize=(12,5))

    for i in range(8):
        plt.subplot(2, 4, i + 1)
        img = plt.imread(DATA_DIR + "/train_images/"+ random_images[i])
        plt.imshow(img)
        plt.axis('off')

    plt.tight_layout()

In [ ]:
# label 0
showImages(label0_images)

In [ ]:
# label 1
showImages(label1_images)

In [ ]:
# label 2
showImages(label2_images)

In [ ]:
# label 3
showImages(label3_images)

In [ ]:
# label 4
showImages(label4_images)

## 各画像に対してのラベル比率の確認

In [ ]:
sns.distplot(train['label'], kde=False)

# 3. モデリング

## データ分割方法の定義

In [ ]:
if CFG.debug:
    train = train[:3000]

train['label'] = train['label'].astype(str)
folds = train.copy()
Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[CFG.target_col])):
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)
print(folds.groupby(['fold', CFG.target_col]).size())

In [ ]:
folds.head()

## データ前処理の定義

In [ ]:
# https://keras.io/ja/preprocessing/image/
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
generator = train_datagen.flow_from_dataframe(dataframe = train,
                                              directory = DATA_DIR + "train_images",
                                              x_col = 'image_id',
                                              y_col = 'label',
                                              target_size = (CFG.size, CFG.size),
                                              color_mode = "rgb",
                                              class_mode = "categorical",
                                              batch_size = 1,
                                              shuffle = True,
                                              subset = 'training')

In [ ]:
# ジェネレーターで8枚生成して、表示する。
plt.figure(figsize=(10, 5))
for i in range(8):
    batches = next(generator)  # (NumBatches, Height, Width, Channels) の4次元データを返す。
    # 画像として表示するため、3次元データにする。
    gen_img = batches[0][0]

    plt.subplot(2, 4, i + 1)
    plt.imshow(gen_img)
    plt.axis('off')
plt.tight_layout()
plt.show()

## 分類モデルの定義

In [ ]:
def vgg16_model(num_classes=None):

    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(CFG.size, CFG.size, 3), pooling='avg')
    output = Dense(num_classes, activation='softmax')(base_model.output)
    model = Model(base_model.input, output)
    
    return model

In [ ]:
model=vgg16_model(CFG.target_size)
model.summary()

## モデルの学習

In [ ]:
def train_loop(folds, fold):

    print(f"========== fold: {fold} training ==========")

    # ====================================================
    # train data, validation data
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)

    # ====================================================
    # generator
    # ====================================================
    
    train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_dataframe(dataframe = train_folds,
                                                        directory = DATA_DIR + "train_images",
                                                        x_col = 'image_id',
                                                        y_col = 'label',
                                                        target_size = (CFG.size, CFG.size),
                                                        color_mode = "rgb",
                                                        class_mode = "categorical",
                                                        batch_size = CFG.batch_size,
                                                        shuffle = True)

    val_generator = val_datagen.flow_from_dataframe(dataframe = valid_folds,
                                                    directory = DATA_DIR + "train_images",
                                                    x_col = 'image_id',
                                                    y_col = 'label',
                                                    target_size = (CFG.size, CFG.size),
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    batch_size = CFG.val_batch_size,
                                                    shuffle = False)
 
    # ====================================================
    # callbacks
    # ====================================================  
    save_model = tf.keras.callbacks.ModelCheckpoint(
                    f'fold-{fold}.h5', monitor='val_loss', verbose=0, save_best_only=True,
                    save_weights_only=True, mode='min', save_freq='epoch')

    
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5, min_delta = 0.0001,
                                                      verbose=1, mode='min'),
    
    reduce_lr_op = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2,
                                     verbose = 1, min_delta = 0.0001, mode = 'min')
    
    callbacks = [save_model, early_stopping, reduce_lr_op]

    # ====================================================
    # optimizer #SGD
    # ====================================================   
    optimizer = Adam(lr=0.0001)
    
    # ====================================================
    # loss
    # ====================================================
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = False, label_smoothing=0.0001,name='categorical_crossentropy' )
    
    # ====================================================
    # model
    # ====================================================
    model = vgg16_model(num_classes=CFG.target_size)    
    model.compile(optimizer = optimizer, loss = loss, metrics = ['categorical_accuracy'])

    # ====================================================
    # training
    # ====================================================
    train_steps = train_generator.n//train_generator.batch_size
    val_steps = val_generator.n//val_generator.batch_size
    
    
    history = model.fit(
                    train_generator,
                    steps_per_epoch=train_steps,
                    epochs=CFG.epochs,
                    validation_data=val_generator,
                    callbacks=callbacks,
                    validation_steps=val_steps
                    )

    # ====================================================
    # predict
    # ====================================================
    print('Loading best model...')
    model.load_weights(f'fold-{fold}.h5')
    
    print('Predicting OOF...')
    pred = model.predict(val_generator, verbose=1)
    
    valid_folds[[str(c) for c in range(5)]] = pred
    valid_folds['preds'] = pred.argmax(1) 
    
    # ====================================================
    # plot
    # ====================================================   
    plt.figure(figsize=(10,3))
    plt.plot(np.arange(len(history.history['categorical_accuracy'])),history.history['categorical_accuracy'],'-o',label='Train ACC',color='#ff7f0e')
    plt.plot(np.arange(len(history.history['val_categorical_accuracy'])),history.history['val_categorical_accuracy'],'-o',label='Val ACC',color='#1f77b4')
    x = np.argmax( history.history['val_categorical_accuracy'] ); y = np.max( history.history['val_categorical_accuracy'] )
    xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max acc\n%.2f'%y,size=14)
    plt.ylabel('ACC',size=14); plt.xlabel('Epoch',size=14)
    plt.legend(loc=2)
    plt2 = plt.gca().twinx()
    plt2.plot(np.arange(len(history.history['loss'])),history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
    plt2.plot(np.arange(len(history.history['val_loss'])),history.history['val_loss'],'-o',label='Val Loss',color='#d62728')
    x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
    ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
    plt.ylabel('Loss',size=14)
    plt.title(f'FOLD {fold} - Image Size {CFG.size}',size=16)
    plt.legend(loc=3)
    plt.show()

    return valid_folds

In [ ]:
folds.head()

In [ ]:
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)


def get_result(result_df):
    preds = result_df['preds'].values
    labels = result_df[CFG.target_col].astype(int).values
    score = get_score(labels, preds)
    print(f'Score: {score:<.5f}')


# train 
oof_df = pd.DataFrame()
for fold in range(CFG.n_fold):
    if fold in CFG.trn_fold:
        _oof_df = train_loop(folds, fold)
        oof_df = pd.concat([oof_df, _oof_df])
        print(f"========== fold: {fold} result ==========")
        get_result(_oof_df)
# CV result
print(f"========== CV ==========")
get_result(oof_df)
# save result
oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)